In [1]:
import os,sys
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch
import shutil
import time
import importlib

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import modules
from bivme.preprocessing.dicom.select_views import select_views
from bivme.preprocessing.dicom.segment_views import segment_views
from bivme.preprocessing.dicom.generate_contours import generate_contours
from bivme.preprocessing.dicom.export_gp import export_gp

nnUNet_raw is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.
nnUNet_results is not defined and nnU-Net cannot be used for training or inference. If this is not intended behavior, please read documentation/setting_up_paths.md for information on how to set this up.


In [3]:
# Check if GPU is available (torch)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

Using device:  cuda


In [96]:
batch_ID = 'fimh-2024-3d' # This will serve as your output folder name
analyst_id = 'jdil469'

In [205]:
src = r"C:\Users\jdil469\bivme-data\fitting\raw-dicoms" + "\\" + 'fimh-2024'
dst = r"C:\Users\jdil469\bivme-data\fitting\processed" + "\\" + batch_ID
states = r"C:\Users\jdil469\bivme-data\fitting\states" + "\\" + batch_ID
output = r"C:\Users\jdil469\bivme-data\fitting\output" + "\\" + batch_ID
model = "models/"

caselist = os.listdir(src)

os.makedirs(dst, exist_ok=True)
os.makedirs(states, exist_ok=True)
os.makedirs(output, exist_ok=True)

# analyst_id = input('Enter analyst ID: ')
case = input('Enter case ID: ')

case_src = os.path.join(src, case)
case_dst = os.path.join(dst, case)

if not os.path.isdir(case_src):
    print(f'Processing case: {case}')
if os.path.exists(case_dst):
    enter = input(f'Case {case} already processed. Do you want to overwrite? (y/n): ')
    if enter == 'y':
        shutil.rmtree(case_dst)
    else:
        print(f'Case {case} skipped.')
        
start_time = time.time()
# Create log file to record some details
states = os.path.join(states, case, analyst_id)
os.makedirs(states, exist_ok=True)
if not os.path.exists(os.path.join(states, 'logs')):
    os.mkdir(os.path.join(states, 'logs'))

log_name = f'{case}_{analyst_id}_{time.ctime().replace(" ","-").replace(":","-")}.txt'
log_path = os.path.join(states, 'logs', log_name)
with open(log_path, 'w') as f:
    f.write(f'Case: {case}\n')
    f.write(f'Batch ID: {batch_ID}\n')
    f.write(f'Source: {case_src}\n')
    f.write(f'Destination: {dst}\n')
    f.write(f'Analyst ID: {analyst_id}\n')
    f.write(f'Processing started at: {time.ctime()}\n')
    f.write('\n')

print(f'Processing case: {case}')


Processing case: cardiohance_120


In [206]:
importlib.reload(sys.modules[select_views.__module__])
from select_views import select_views

In [207]:
# option = 'default'
option = 'load'

# Temporarily displace states folder to 2d to load view predictions
states_2d = r"C:\Users\jdil469\bivme-data\fitting\states\fimh-2024-2d"
states_2d = os.path.join(states_2d, case, analyst_id)

slice_info_df, num_phases, slice_mapping = select_views(case, case_src, case_dst, model, states_2d, option)
print('View selection complete.')

print(f'Number of phases: {num_phases}')

Loading view predictions from states folder...
Data prepared for view prediction. 14 image series found.
View predictions for cardiohance_120:
SAX-atria: 1 series
SAX: 5 series
4ch: 1 series
Excluded: 1 series
3ch: 1 series
2ch: 1 series
2ch-RT: 1 series
LVOT: 1 series
RVOT: 1 series
RVOT-T: 1 series
View selection complete.
Number of phases: 20


In [208]:
importlib.reload(sys.modules[segment_views.__module__])
from segment_views import segment_views

In [209]:
## Segmentation
version = '3d' # 2d or 3d
start_time = time.time()
segment_views(case, dst, model, slice_info_df, version = version)
end_time = time.time()
print(f'Segmentation complete. Time taken: {end_time-start_time} seconds ({version} version).')

# Add segmentation time to log
with open(log_path, 'a') as f:
    f.write(f'Segmentation time: {end_time-start_time} seconds ({version} version).\n')


Writing SAX images to nifti files...
Segmenting SAX images...
*** Making predictions for SAX images ***
There are 5 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 5 cases that I would like to predict

Predicting SAX_3d_10:
perform_everything_on_device: True


100%|██████████| 2/2 [00:01<00:00,  1.10it/s]


sending off prediction to background worker for resampling and export
done with SAX_3d_10

Predicting SAX_3d_11:
perform_everything_on_device: True


100%|██████████| 2/2 [00:01<00:00,  1.36it/s]


sending off prediction to background worker for resampling and export
done with SAX_3d_11

Predicting SAX_3d_7:
perform_everything_on_device: True


...

sending off prediction to background worker for resampling and export
done with 3ch_3d_14
Done with 3ch

Writing 4ch images to nifti files...
Segmenting 4ch images...
*** Making predictions for 4ch images ***
There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting 4ch_3d_12:
perform_everything_on_device: True


100%|██████████| 4/4 [00:03<00:00,  1.24it/s]


sending off prediction to background worker for resampling and export
done with 4ch_3d_12
Done with 4ch

Writing RVOT images to nifti files...
Segmenting RVOT images...
*** Making predictions for RVOT images ***
There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting RVOT_3d_18:
perform_everything_on_device: True


100%|██████████| 2/2 [00:01<00:00,  1.43it/s]


sending off prediction to background worker for resampling and export
done with RVOT_3d_18
Done with RVOT

Segmentation complete. Time taken: 86.49694514274597 seconds (3d version).


In [210]:
# Reload
importlib.reload(sys.modules[generate_contours.__module__])
from generate_contours import generate_contours

In [211]:
slice_dict = generate_contours(case, dst, slice_info_df, num_phases, version = version)
print('Guide points generated.')

Generating contours for SAX slice 7...
Generating contours for SAX slice 8...
Generating contours for SAX slice 9...
Generating contours for SAX slice 10...
Generating contours for SAX slice 11...
Generating contours for 2ch slice 15...
Generating contours for 3ch slice 14...
Generating contours for 4ch slice 12...
Generating contours for RVOT slice 18...
Guide points generated.


In [212]:
# Reload
importlib.reload(sys.modules[export_gp.__module__])
from export_gp import export_gp

In [213]:
export_gp(case, dst, output, slice_dict, slice_mapping)
print('Export complete.')
print(f'Case {case} complete.')
print(f'Total time taken: {time.time()-start_time} seconds.')

Export complete.
Case cardiohance_120 complete.
Total time taken: 92.13595986366272 seconds.
